In [1]:
import pandas as pd
import numpy as np

# I - Consolidations of datasets

In [2]:
patients = [103001, 111001, 113001, 124001]

df_ml_conso = pd.read_csv('../../datasets/2_dataset_creation/df_ml_{}.csv'.format(patients[0]))
df_ml_conso['patient'] = patients[0]

for patient in patients[1:]:
    df_temp = pd.read_csv('../../datasets/2_dataset_creation/df_ml_{}.csv'.format(patient))
    df_temp['patient'] = patient
    df_ml_conso = pd.concat([df_ml_conso,df_temp], axis=0)

# Removing timestamp column
df_ml_conso = df_ml_conso.iloc[:,1:]

display(df_ml_conso.head())

print('\nPatient count by classification:')
display(pd.pivot_table(df_ml_conso, values=['classif'], index='patient',aggfunc='count'))

print('\nPatient average by classification:')
display(pd.pivot_table(df_ml_conso, values=['classif', 'classif_avg'], index='patient',aggfunc=np.mean))

,basSQI_score,cSQI_score,kSQI_score,pSQI_score,qSQI_score,sSQI_score,classif,classif_avg,patient
0,0.93,0.63,13.47,0.52,0.88,3.34,1,1.0,103001
1,0.93,0.66,13.21,0.52,0.93,3.30,1,1.0,103001
2,0.94,0.70,13.18,0.53,0.93,3.30,1,1.0,103001
3,0.92,0.54,12.14,0.52,1.00,3.18,1,1.0,103001
4,0.92,0.57,12.76,0.51,1.00,3.24,1,1.0,103001



Patient count by classification:


,classif
patient,
103001,267
111001,10071
113001,400
124001,533



Patient average by classification:


,classif,classif_avg
patient,,
103001,0.558052,0.723402
111001,0.348128,0.419476
113001,0.520000,0.659677
124001,0.110694,0.287970


# II - Appying threshold of quality

In [3]:
# Setting a threshold : proportion of optimal by observation
classif_threshold = 0.95

df_ml_conso['classif_threshold'] = df_ml_conso['classif_avg'].apply(lambda x: 1 if x >= classif_threshold else 0)

print('\nPatient count:')
display(pd.pivot_table(df_ml_conso, values=['classif'], index='patient',aggfunc='count'))

print('\nPatient average by classification:')
display(pd.pivot_table(df_ml_conso, values=['classif', 'classif_avg', 'classif_threshold'], index='patient',aggfunc=np.mean))


Patient count:


,classif
patient,
103001,267
111001,10071
113001,400
124001,533



Patient average by classification:


,classif,classif_avg,classif_threshold
patient,,,
103001,0.558052,0.723402,0.561798
111001,0.348128,0.419476,0.353490
113001,0.520000,0.659677,0.532500
124001,0.110694,0.287970,0.129456


# III - Equalisation of repartition by patient

In [4]:
df_ml_conso_balanced = pd.DataFrame()

for patient in patients:
    df_class1 = df_ml_conso[(df_ml_conso['patient'] == patient) & (df_ml_conso['classif_threshold'] ==1)]
    df_class0 = df_ml_conso[(df_ml_conso['patient'] == patient) & (df_ml_conso['classif_threshold'] ==0)]

    if df_class1.shape[0] >= df_class0.shape[0]:
        df_ml_conso_balanced = pd.concat([df_ml_conso_balanced,
                              df_class0,
                              df_class1.sample(df_class0.shape[0])]
                            )
    else:
        df_ml_conso_balanced = pd.concat([df_ml_conso_balanced,
                        df_class0.sample(df_class1.shape[0]),
                        df_class1]
                    )

print('\nPatient count:')
display(pd.pivot_table(df_ml_conso_balanced, values=['classif'], index='patient',aggfunc='count'))

print('\nPatient average by classification:')
display(pd.pivot_table(df_ml_conso_balanced, values=['classif', 'classif_avg', 'classif_threshold'], index='patient',aggfunc=np.mean))

print('\nProportion of each class')
print(df_ml_conso_balanced['classif_threshold'].value_counts())


Patient count:


,classif
patient,
103001,234
111001,7120
113001,374
124001,138



Patient average by classification:


,classif,classif_avg,classif_threshold
patient,,,
103001,0.495726,0.684395,0.5
111001,0.492416,0.553600,0.5
113001,0.486631,0.636018,0.5
124001,0.427536,0.605152,0.5



Proportion of each class
1    3933
0    3933
Name: classif_threshold, dtype: int64


# IV - Export

In [5]:
df_ml_conso_balanced = df_ml_conso_balanced.drop(['patient', 'classif', 'classif_avg'], axis=1)
df_ml_conso_balanced.rename(columns={df_ml_conso_balanced.columns[-1]: "classification"}, inplace=True)
display(df_ml_conso_balanced.head())

,basSQI_score,cSQI_score,kSQI_score,pSQI_score,qSQI_score,sSQI_score,classification
19,0.83,0.60,6.17,0.50,0.57,-0.35,0
20,0.94,0.59,10.79,0.51,0.85,2.97,0
67,0.91,0.53,9.13,0.53,0.80,2.50,0
68,0.79,0.62,1.96,0.51,0.79,0.93,0
133,0.96,0.55,16.03,0.52,0.87,3.69,0


In [6]:
df_ml_conso_balanced.to_csv('../../datasets/3_ml_patients_consolidation/df_ml_conso_balanced.csv', index=False)